In [1]:
import numpy as np
import cv2
import os
import statistics
from scipy.stats import pearsonr
from PIL import Image
from sklearn.metrics import mean_squared_error

In [2]:
data_dir = 'E:/allen'
data_class = os.listdir(data_dir)
test_dir = 'E:/cgan/model/enc/imgs_unet_guide_test'
class_dir = os.listdir(test_dir)
num_examples = []
for protein in data_class:
    protein_folder = os.path.join(data_dir, protein)
    num_examples.append(len(os.listdir(protein_folder)))

In [3]:
count = 0
mean_pearson = []
mean_rmse = []
total_examples = []
training_examples = []
test_examples = []
for folder in class_dir:
    class_folder = os.path.join(test_dir, folder)
    print('{}: {}'.format(folder, data_class[count]))
    pearson_scores = []
    rmse_scores = []
    for filename in os.listdir(class_folder):
        img_folder = os.path.join(class_folder, filename)
        for image in os.listdir(img_folder):
            if image == 'ground_truth.png':
                ground_truth_path = os.path.join(img_folder, image)
                ground_truth = Image.open(ground_truth_path)
                ground_truth = np.asarray(ground_truth)
                prot_truth = ground_truth[:,:,1]
                res = np.zeros((256, 256, 3))
                res[:,:,1] = ground_truth[:,:,1]
                res_file = os.path.join(img_folder, 'prot_truth.png')
                cv2.imwrite(res_file, res)
                prot_truth = prot_truth.flatten()
            elif image == 'prediction.png':
                prediction_path = os.path.join(img_folder, image)
                prediction = Image.open(prediction_path)
                prediction = np.asarray(prediction)
                prot_pred = prediction[:,:,1]
                prot_pred = prot_pred.flatten()
        pred_img = np.zeros((256,256,3))
        pred_img[:,:,0] = ground_truth[:,:,2]
        pred_img[:,:,1] = prediction[:,:,1]
        pred_img[:,:,2] = ground_truth[:,:,0]
        pred_img_file = os.path.join(img_folder, 'pred_image.png')
        cv2.imwrite(pred_img_file, pred_img)
        pearson_value = pearsonr(prot_truth, prot_pred)
        pearson_scores.append(pearson_value[0])
        rmse = mean_squared_error(prot_truth, prot_pred, squared=False)
        rmse_scores.append(rmse)
    mean_pearson.append(statistics.mean(pearson_scores))
    mean_rmse.append(statistics.mean(rmse_scores))
    total_examples.append(num_examples[count])
    training_examples.append(num_examples[count] - len(os.listdir(class_folder)))
    test_examples.append(len(os.listdir(class_folder)))
    print('----Statistics----')
    print('Average of Pearson Correlation: {}'.format(statistics.mean(pearson_scores)))
    print('Standard Deviation of Pearson Correlation: {}'.format(statistics.stdev(pearson_scores)))
    print('Median of Pearson Correlation: {}'.format(statistics.median(pearson_scores)))
    print('Average of RMSE: {}'.format(statistics.mean(rmse_scores)))
    print('Standard Deviation of RMSE: {}'.format(statistics.stdev(rmse_scores)))
    print('----Data----')
    print('Total Number of Examples: {}'.format(num_examples[count]))
    print('Number of Training Examples: {}'.format(num_examples[count] - len(os.listdir(class_folder))))
    print('Number of Test Examples: {}'.format(len(os.listdir(class_folder))))
    print('%'*40)
    count += 1

cls0: Alpha actinin
----Statistics----
Average of Pearson Correlation: 0.7646774219614801
Standard Deviation of Pearson Correlation: 0.06372878138820261
Median of Pearson Correlation: 0.7711706019949779
Average of RMSE: 2.2599999761662573
Standard Deviation of RMSE: 0.2760167782262204
----Data----
Total Number of Examples: 493
Number of Training Examples: 416
Number of Test Examples: 77
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
cls1: Alpha tubulin
----Statistics----
Average of Pearson Correlation: 0.8266765750554782
Standard Deviation of Pearson Correlation: 0.10745356165691103
Median of Pearson Correlation: 0.8738047327068581
Average of RMSE: 2.2713183683489517
Standard Deviation of RMSE: 0.28296649338036955
----Data----
Total Number of Examples: 1043
Number of Training Examples: 873
Number of Test Examples: 170
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
cls2: Beta actin
----Statistics----
Average of Pearson Correlation: 0.8931484827344026
Standard Deviation of Pearson Correlation: 0.052

In [24]:
print(mean_pearson)

[0.777148733538518, 0.8336430488558744, 0.8933665009604649, 0.7343321235325144, 0.5724404582256654, 0.6602404789868863, 0.9218844871159495, 0.4802497205572959, 0.8689638464970421, 0.6700321043309736]


In [4]:
import csv
with open('data.csv', 'w', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    datawriter.writerow([data_class[i] for i in range(10)])
    datawriter.writerow([class_dir[i] for i in range(10)])
    datawriter.writerow([mean_pearson[i] for i in range(10)])
    datawriter.writerow([mean_rmse[i] for i in range(10)])
    datawriter.writerow([total_examples[i] for i in range(10)])
    datawriter.writerow([training_examples[i] for i in range(10)])
    datawriter.writerow([test_examples[i] for i in range(10)])